In [1]:
import logging
import pandas as pd
from py4f_utils import py4f_utils
import matplotlib.pyplot as plt
import psycopg2
import psycopg2.extras as extras




# Define the ETF ticker symbol
ticker = 'SPY'

# Fetch historical data for the month of March
march_start = '2024-03-01'
march_end = '2024-03-31'
march_data = py4f_utils.getDataFromYf(ticker, start=march_start, end=march_end)

if march_data is not None:
    logging.info("Successfully retrieved historical data for March.")

# Fetch historical data from March to the current date
current_date = pd.Timestamp.now().strftime('%Y-%m-%d')
march_to_current_data = py4f_utils.getDataFromYf(ticker, start=march_start, end=current_date)

if march_to_current_data is not None:
    logging.info("Successfully retrieved historical data from March to current date.")


# Database connection parameters
conn_params = {
    'dbname': 'pythonForFinanceDb',
    'user': 'postgres',
    'password': 'password',
    'host': 'localhost',
    'port': 5432
}

# Establish the connection
conn = psycopg2.connect(**conn_params)

# Define the function for bulk insert
def execute_values(conn, df, table, ticker):
    """
    Using psycopg2.extras.execute_values() to insert or update the dataframe
    """
    # Create a list of tuples from the dataframe values
    tuples = [tuple([ticker] + list(x)) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = 'ticker,' + ','.join(list(df.columns))
    # SQL query to execute
    query  = f"INSERT INTO {table}({cols}) VALUES %s ON CONFLICT (ticker, date) DO UPDATE SET "
    update_cols = ','.join([f"{col}=EXCLUDED.{col}" for col in list(df.columns)[1:]])
    query += update_cols
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

# exit(1)
# # Display the first few rows of the data
# print(hist.head())
# # View summary statistics of the DataFrame
# print(hist.describe())

# # Plot the closing prices over time using the Date column as x-axis
# plt.figure(figsize=(10, 5))
# plt.plot(hist['Date'], hist['Close'], label='Closing Price')
# plt.title('TSLA Closing Prices')
# plt.xlabel('Date')
# plt.ylabel('Close Price')
# plt.legend()
# plt.show()

[*********************100%%**********************]  1 of 1 completed
INFO:root:Successfully retrieved historical data for March.
[*********************100%%**********************]  1 of 1 completed
INFO:root:Successfully retrieved historical data from March to current date.


execute_values() done


In [3]:
# data1 = py4f_utils.getDataFromYf(ticker)
conn = psycopg2.connect(**conn_params)
try:
    execute_values(conn, march_to_current_data, 'stock_data', ticker)
finally:
    # Close the connection
    conn.close()

Error: duplicate key value violates unique constraint "stock_data_ticker_date_key"
DETAIL:  Key (ticker, date)=(SPY, 2024-03-01) already exists.

